In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from multiprocessing import Pool

c:\Programs\playwright-learning\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
actual_data = pd.read_csv('glassdoordata.csv',sep='\t')
actual_data.head(2)

,conm,gvkey,tic,cik,cusip
0,AAR CORP,1004,AIR,1750.0,000361105
1,ADC TELECOMMUNICATIONS INC,1013,ADCT.1,61478.0,000886309


In [4]:
glassdoor_data = pd.read_excel('./backup/unique-data-glassdoor-companies-list.xlsx')
glassdoor_data.head(2)

,CompanyID,CompanyName,CompanySize
0,740850,TESLA,"{""filterType"": ""EMPLOYEES_COUNT"", ""minInclusiv..."
1,2099435,Courtyard,"{""filterType"": ""EMPLOYEES_COUNT"", ""minInclusiv..."


In [5]:


def calculate_similarity(args):
    name1, gvkey, tic, cik, cusip, row2 = args
    return (name1, gvkey, tic, cik, cusip, row2['CompanyName'], row2['CompanyID'], fuzz.ratio(name1, row2['CompanyName']))

# Create an empty list to store mapping results
mapped_companies = []

# Define a function to calculate similarity for a row in df1
def calculate_similarity_for_row(row1, df2):
    pool_args = [(row1['conm'], row1['gvkey'], row1['tic'], row1['cik'], row1['cusip'], row2) for _, row2 in df2.iterrows()]
    with Pool() as pool:
        results = pool.map(calculate_similarity, pool_args)
    return results

# Iterate through the rows in df1 and find the best match in df2
for _, row1 in actual_data.iterrows():
    results = calculate_similarity_for_row(row1, glassdoor_data)
    best_match = max(results, key=lambda x: x[2], default=(row1['conm'], row1['gvkey'], row1['tic'], row1['cik'], row1['cusip'], row2['CompanyID'], 'Not Found', 0))
    print(best_match)
    # You can set a threshold score to consider a match or adjust it as needed
    if best_match[2] >= 80:
        mapped_companies.append(best_match)
    else:
        mapped_companies.append((row1['conm'], row1['gvkey'], row1['tic'], row1['cik'], row1['cusip'], 'Not Found', 0))

mapped_df = pd.DataFrame(mapped_companies, columns=['conm', 'gvkey', 'tic', 'cik', 'cusip', 'CompanyName', 'CompanyID' 'SimilarityScore'])


In [ ]:
print(mapped_companies)